In [1]:
import pandas as pd
import os
import random
import json
from datetime import datetime
import re
import pandas as pd
from utils import *
from collections import defaultdict

In [3]:
experiment_folders = []
output_path = 'output/gpt-3_5-turbo-games/annotated/our_annotations/'
# output_path = 'output/gpt-3_5-turbo-games/annotated/'
# output_path = 'output/gpt-4-games/annotated/'
for x in os.listdir(output_path):
    if x.startswith('game_') and os.path.isdir(os.path.join(output_path, x)):
        experiment_folders.append(output_path + x)

In [4]:
len(experiment_folders)

5

In [5]:
impactful_intents = {
    'cast suspicion on innocent players': True, 
    'counter the team proposal because you think a player on the team is evil': True, 
    'express concerns about a player from a failed quest team and suggest to not include them in the current team': True, 
    'express confusion for failure of mission': True, 
    'express disagreement and vote disagree only if you are not in the proposed team': True, 
    'express you are a loyal player': True, 
    'pretend to be merlin by providing hints on who is evil': True, 
    'question a player about why they changed their statements': True, 
    'question a player on why they didnt vote for the last team': True, 
    'question the leader why they selected a particular player for the team': True, 
    'reveal identity by telling others you are servant to persuade other loyal players': True, 
    'share concerns about an evil player': True, 
    'strongly convince the team leader to choose you on team': True, 
    'support loyal players of the previous quest team if the mission failed': True, 
    'support your teammate by expressing that he is good': True
}

In [6]:
context_questions = [
    "If there have been discussions before your turn, what discussions have happened in this round?",
    "After analyzing the responses to the previous question, what do you need to do to win from here?"
]

In [7]:
role_details = {
    "Merlin": """You are Merlin. These are your goals and characteristics: 
- Know the identities of evil players.
- Subtly guide your team, especially Percival.
- Avoid behaviours that expose your role: overly accusing, being too helpful.
- Goal(Main Intent): Win without revealing identity.""",

    "Percival": """You are percival and you need to help Merlin. These are your goals and characteristics:
- Know identities of Merlin and Morgana, but unsure who is who.
- Use subtle hints to guide team and protect Merlin.
- Be cautious not to expose Merlin while deciphering true identities.
- Goal(Main Intent): Win while safeguarding Merlin.""",

    "Servant": """You are Loyal Servant of Arthur. These are your goals and characteristics: 
- No special knowledge, rely on discussion and voting.
- Contribute to the success of Quests
- Goal(Main Intent): Win by helping complete Quests and protecting Merlin.""",

    "Assassin": """You are assassin. These are your goals and characteristics:
- Discreetly seek Merlin's identity.
- Work to prevent Quests success.
- Goal(Main Intent): Win either by failing quests or assassinating Merlin.""",

    "Morgana": """You are Morgana. These are your goals and characteristics: 
- Pretend to be Merlin to mislead Percival and the good side.
- Goal(Main Intent): Fail quests and Confuse and sabotage to win."""
}

In [8]:
def get_role_extra_info(role, chart_to_player):
    if role == "Merlin":
        return f"{chart_to_player['Morgana']} and {chart_to_player['Assassin']} are evil players"
    elif role == "Percival":
        return f"{chart_to_player['Merlin']} and {chart_to_player['Morgana']} are Merlin and Morgana, but you don't know which is Merlin and which is Morgana."
    elif role == "Servant":
        return ""
    elif role == "Morgana":
        return f"Assassin is {chart_to_player['Assassin']}"
    return f"Morgana is {chart_to_player['Morgana']}"

In [9]:
context_template = """\
**Name**: {}

**Role**: {}

**Role Details**: 
{}

**Round**: {}

**Current Leader**: {}

**Current Team**: {}

**Previous Rounds Team Voting**:
{}

**Previous Results**: 
{}

**Previous Rounds Summary**:
{}
"""

reasonable_intent_context_template_extn = """\

**Previous Discussions(in the current round)**: 
{}
"""

In [10]:
intent_guessing_context_template = """\
**Name**: {}

**Role**: {}

**Role Details**: 
{}

**Round**: {}

**Speaker Name**: {}

**Current Leader**: {}

**Current Team**: {}

**Previous Rounds Team Voting**:
{}

**Previous Results**: 
{}

**Previous Rounds Summary**:
{}
"""

In [11]:
second_order_intent_guessing_context_template = """\
**Name**: {}

**Role**: {}

**Role Details**: 
{}

**Round**: {}

**Player Name**: {}

**Current Leader**: {}

**Current Team**: {}

**Previous Rounds Team Voting**:
{}

**Previous Results**: 
{}

**Previous Rounds Summary**:
{}
"""

In [12]:
def format_context(data, include_discussion=False):
    char_to_player = data['char_to_player']

    other_players = ", ".join([f"{val} = {key}" for key, val in sorted(char_to_player.items(), key=lambda x: x[1]) if val != data['name']])
    previous_results = "No previous results."
    if len(data['previous_results']) > 0:
        previous_results = '\n'.join([f"Round {key}: Team = {', '.join(val['team'])}, Result = {val['result']}" for key, val in data['previous_results'].items()])

    previous_votes = "No previous team votes. This is the first round."
    if len(data['previous_votes']) > 0:
        previous_votes = data['previous_votes']

    previous_summaries = "No previous summaries. This is the first round."
    if len(data['previous_summaries']) > 0:
        previous_summaries = data['previous_summaries']

    
    role_extra_info = get_role_extra_info(data['role'], char_to_player)
    role_details_enriched = data['role_details'] + '\n- ' + role_extra_info

    context = context_template.format(
        data['name'],
        data['role'],
        role_details_enriched,
        # other_players,
        int(data['round'])%10,
        data['round_leader'],
        ', '.join(data['round_team']),
        previous_votes,
        previous_results,
        previous_summaries
    )

    if include_discussion:
        previous_discussions = "No previous discussions."
        if len(data['previous_discussions']) > 0:
            previous_discussions = data['previous_discussions']
        context += reasonable_intent_context_template_extn.format(previous_discussions)

    return context

In [13]:
def format_context_for_intent_guessing(data, include_discussion=False):
    # char_to_player = data['char_to_player']

    # other_players = ", ".join([f"{val} = {key}" for key, val in sorted(char_to_player.items(), key=lambda x: x[1]) if val != data['name']])
    previous_results = "No previous results."
    if len(data['previous_results']) > 0:
        previous_results = '\n'.join([f"Round {key}: Team = {', '.join(val['team'])}, Result = {val['result']}" for key, val in data['previous_results'].items()])

    previous_votes = "No previous team votes. This is the first round."
    if len(data['previous_votes']) > 0:
        previous_votes = data['previous_votes']

    previous_summaries = "No previous summaries. This is the first round."
    if len(data['previous_summaries']) > 0:
        previous_summaries = data['previous_summaries']

    
    role_details_player = data['role_details']

    context = intent_guessing_context_template.format(
        data['name'],
        data['role'],
        role_details_player,
        int(data['round'])%10,
        data['speaker_name'],
        data['round_leader'],
        ', '.join(data['round_team']),
        previous_votes,
        previous_results,
        previous_summaries
    )

    if include_discussion:
        previous_discussions = "No previous discussions."
        if len(data['previous_discussions']) > 0:
            previous_discussions = data['previous_discussions']
        context += reasonable_intent_context_template_extn.format(previous_discussions)

    return context

In [14]:
def format_context_for_second_order_intent_guessing(data, include_discussion=False):
    # char_to_player = data['char_to_player']

    # other_players = ", ".join([f"{val} = {key}" for key, val in sorted(char_to_player.items(), key=lambda x: x[1]) if val != data['name']])
    previous_results = "No previous results."
    if len(data['previous_results']) > 0:
        previous_results = '\n'.join([f"Round {key}: Team = {', '.join(val['team'])}, Result = {val['result']}" for key, val in data['previous_results'].items()])

    previous_votes = "No previous team votes. This is the first round."
    if len(data['previous_votes']) > 0:
        previous_votes = data['previous_votes']

    previous_summaries = "No previous summaries. This is the first round."
    if len(data['previous_summaries']) > 0:
        previous_summaries = data['previous_summaries']

    
    role_details_speaker = data['speaker_role_details']

    context = second_order_intent_guessing_context_template.format(
        data['speaker_name'],
        data['speaker_role'],
        role_details_speaker,
        int(data['round'])%10,
        data['name'],
        data['round_leader'],
        ', '.join(data['round_team']),
        previous_votes,
        previous_results,
        previous_summaries
    )

    if include_discussion:
        previous_discussions = "No previous discussions."
        if len(data['previous_discussions']) > 0:
            previous_discussions = data['previous_discussions']
        context += reasonable_intent_context_template_extn.format(previous_discussions)

    return context

In [15]:
player_order = {
    '1': [1, 2, 3, 4, 5],
    '2': [2, 3, 4, 5, 1],
    '3': [3, 4, 5, 1, 2],
    '4': [4, 5, 1, 2, 3],
    '5': [5, 1, 2, 3, 4]
}

In [16]:
players = ['Player' + str(num) for num in range(1, 6)]
rounds = [str(num) for num in range(1, 6)]
rounds.extend([str(num) for num in range(11, 16)])
rounds

['1', '2', '3', '4', '5', '11', '12', '13', '14', '15']

In [17]:
def get_previous_discussions(cur_player, cur_rnd, data, round_leader):
    player_id = int(cur_player.replace('Player', ''))
    round_leader_id = round_leader.replace('Player', '')

    rnd = int(cur_rnd)
    # if rnd > 10:
    #     rnd = rnd%10 + 1
    #     rnd%5

    rnd = str(rnd)
    idx = 0
    for i, val in enumerate(player_order[round_leader_id]):
        if val == player_id:
            idx = i
            break


    previous_players = [f'Player{i}' for i in player_order[round_leader_id][:idx]]

    discussions = []
    for player in previous_players:
        discussions.append(f"{player}: {data[player][cur_rnd]['discussion'][0]['speech']}")
    return '\n'.join(discussions)


In [18]:
summary_questions = ["Who was the leader for this round?",
                     "What was the proposed team for this round?",
                     "Why do you think the leader proposed the current team?",
                     "Did everyone agree for the proposed team?",
                     "If some people disagreed, why do you think they disagreed?",
                     "Was the quest successful?",
                     "If the quest was successful, was any evil player part of the team? If yes, then why did they vote success?",
                     "If the quest failed, how do you plan to protect the evil player in the next round?",
                     "What do the evil side need to do from here to win the game?"
                    ]

In [19]:
def get_previous_round_summaries(name, cur_round, data):
    summaries = {}
    cur_round_mod = int(cur_round)%10
    for rnd in rounds:
        if int(rnd)%10 <= cur_round_mod:
            if rnd in data[name] and 'summary' in data[name][rnd]:
                summary = json.loads(data[name][rnd]['summary'][0]['summary'])['Answers']
                summary_txt = ''
                for idx in range(len(summary)):
                    summary_txt += f"{summary[idx]}\n"
                summaries[rnd] = summary_txt
    
    summaries_list = [(round, summaries[round]) for round in sorted(summaries.keys(), key=lambda x: int(x[-1]))]

    disagreement_msg = '(players didn\"t agree on team)'
    res = '\n'.join(f"Round {int(rnd)%10} {disagreement_msg if str(int(rnd)+10) in summaries else ''}: \n{val} \n" for rnd, val in summaries_list)
    sum_q_txt = '\n'.join(summary_questions)
    res = f"Summary questions: \n\n{sum_q_txt}\n\n{res}"
    return res

In [20]:
def check_int_fol(int_fol):
    fol_think, fol_speech = int_fol[0], int_fol[1]
    for a, b in zip(fol_think, fol_speech):
        if a<3 or b<3:
            return False
    return True

In [21]:
def get_summarization_data_point(id, all_intents, formulation_con_data, refinement_con_data, context_intnt_following, int_fol_form, int_fol_ref):
    if len(all_intents["modified"]) == 0 or set(all_intents["original"]) == set(all_intents["modified"]):
            if check_int_fol(int_fol_form):
                id =  id + "_form"
                complete_intent_following = {
                    'id': id,
                    'context': context_intnt_following,
                    'intents': all_intents["original"],
                    'think': formulation_con_data['think'],
                    'speak': formulation_con_data['speech'],
                }
                return [complete_intent_following]
    
    res = []
    if check_int_fol(int_fol_form):
        form_intent_following = {
            'id': id + "_form",
            'context': context_intnt_following,
            'intents': all_intents["original"],
            'think': formulation_con_data['think'],
            'speak': formulation_con_data['speech'],
        }
        res.append(form_intent_following)

    if check_int_fol(int_fol_ref):
        ref_intent_following = {
            'id': id + "_ref",
            'context': context_intnt_following,
            'intents': all_intents["modified"],
            'think': refinement_con_data['think'],
            'speak': refinement_con_data['speech'],
        }
        res.append(ref_intent_following)

    return res

In [22]:
def get_context_for_intent_guessing(context1):
    context = context1.copy()
    character_to_player = context['char_to_player']
    roles = list(character_to_player.keys())
    player_role = context['role']
    player_name = context['name']
    player_role_details = role_details[player_role]

    role_choices = [v for v in roles if v != player_role]
    selected_role = random.choice(role_choices)
    selected_player = character_to_player[selected_role]

    selected_role_details = role_details[selected_role]
    extra_role_info = get_role_extra_info(selected_role, character_to_player)
    selected_role_details = selected_role_details + '\n- ' + extra_role_info

    context['speaker_name'] = player_name
    context['speaker_role'] = player_role
    context['speaker_role_details'] = player_role_details
    context['name'] = selected_player
    context['role'] = selected_role
    context['role_details'] = selected_role_details

    # return format_context_for_intent_guessing(context, include_discussion=True)
    return format_context_for_second_order_intent_guessing(context, include_discussion=True)
        
    

In [23]:
def get_single_round_data(gameplay_id, cur_round, gameplay_data, raw_data, role, name, complete_results, vote_results, int_following_form, int_following_ref):
    intent_data = raw_data['intent'][0]
    intents = intent_data['intents']
    intents = [remove_non_alphabets(intent) for intent in intents]
    reason = intent_data['think']

    round_leader = gameplay_data['round_leaders'][cur_round]

    # discussion_summary = ""
    # if len(intent_data['context_answers']) > 0:
    #     discussion_summary = intent_data['context_answers'][0]

    previous_discussions = get_previous_discussions(name, cur_round, complete_results, round_leader)
    previous_summaries = get_previous_round_summaries(name, cur_round, complete_results)

    formulation_con_data = raw_data['formulation_con'][0]

    modified_intents = intents
    if 'intent_modification' in raw_data:
        intent_modification_data = raw_data['intent_modification'][0]
        modified_intents = intent_modification_data['new_intents']
        modified_intents = [remove_non_alphabets(intent) for intent in modified_intents]

    refinement_con_data = raw_data['discussion'][0]

    quest_results = gameplay_data['team_quest_result']
    

    cur_team = []

    previous_votes = {}
    for rnd in vote_results.keys():
        if int(rnd)%10 <= int(cur_round)%10:
            vote_txt = ', '.join([f"{player} = {vote}" for player, vote in vote_results[rnd].items()])
            previous_votes[rnd] = vote_txt

    previous_votes_list = [(round, previous_votes[round]) for round in sorted(previous_votes.keys(), key=lambda x: int(x[-1]))]

    disagreement_msg = '(players didn\"t agree on team)'
    previous_votes_str = '\n'.join(f"Round {int(rnd)%10}: {val} {disagreement_msg if str(int(rnd)+10) in previous_votes else ''}" for rnd, val in previous_votes_list)

    previous_results = {}
    for rnd in quest_results.keys():
        if int(rnd)%10 < int(cur_round)%10:
            team = list(quest_results[rnd].keys())
            result = 'success'
            if 'failure' in quest_results[rnd].values():
                result = 'failure'
            previous_results[rnd] = {'team': team, 'result': result}
        if int(rnd) == int(cur_round) and name != round_leader:
            cur_team = list(quest_results[rnd].keys())

    context = {
        'char_to_player': gameplay_data['character_to_player'],
        'round': str(int(cur_round)),
        'name': name, 
        'role': role, 
        'role_details': role_details[role],
        'round_leader': round_leader,
        'round_team': cur_team,
        'previous_votes': previous_votes_str,
        'previous_results': previous_results,
        'previous_discussions': previous_discussions,
        'previous_summaries': previous_summaries
        # 'discussion_summary': discussion_summary
    }

    context_intent_guessing = get_context_for_intent_guessing(context)

    context_intnt_following_with_discussion = format_context(context, include_discussion=True)

    context_reasonable = context_intnt_following_with_discussion

    all_intents = {'original': intents, 'modified': modified_intents}

    # intents = "\n".join([f"{num+1}. {intent}" for num, intent in enumerate(intents)])
    # modified_intents = "\n".join([f"{num+1}. {intent}" for num, intent in enumerate(modified_intents)])
    

    id_ = f"{gameplay_id}-{name.lower()}-{cur_round}"

    reasonable_intent = {
        'id': id_,
        'context': context_reasonable,
        'intents': intents,
        'reason': reason
    }

    formulation_con_intent_following = {
        'id': id_,
        'context': context_intnt_following_with_discussion,
        'intents': intents,
        'think': formulation_con_data['think'],
        'speak': formulation_con_data['speech']
    }

    refinement_con_intent_following = {
        'id': id_,
        'context': context_intent_guessing,
        'intents': modified_intents,
        'think': refinement_con_data['think'],
        'speak': refinement_con_data['speech']
    }
    
    
    complete_intent_following = get_summarization_data_point(
        id_, 
        all_intents, 
        formulation_con_data, 
        refinement_con_data, 
        context_intnt_following_with_discussion, 
        int_following_form, 
        int_following_ref
    )

    return reasonable_intent, formulation_con_intent_following, refinement_con_intent_following, complete_intent_following


In [24]:
def get_vote_results(results):
    vote_results = {}
    for rnd in rounds:
        for player in players:
            if rnd in results[player]:
                vote = results[player][rnd]['vote'][0]['vote_result']
                if rnd not in vote_results:
                    vote_results[rnd] = {}
                vote_results[rnd][player] = vote

    return vote_results

In [25]:
reasonable_intent_data = []
form_con_intent_following_data = []
refine_con_intent_following_data = []
complete_intent_following_data = []


# rounds.append('13')

In [26]:
def process_results(gameplay_id, gameplay_data, results, final_player_rounds):
    vote_results = get_vote_results(results)
    for player in players:
        for rnd in rounds:
            if rnd in results[player]:
                is_secondary_round = False
                if int(rnd) > 10:
                    is_secondary_round = True
                key = (player, int(rnd)%10, is_secondary_round)
                if key not in final_player_rounds:
                    continue
                # print(gameplay_id, rnd, player)
                reasonable_intent, form_con_intent_following, refine_con_intent_following, complete_intent_following = get_single_round_data(
                    gameplay_id,
                    rnd, 
                    gameplay_data, 
                    results[player][rnd], 
                    results[player]['role'], 
                    player,
                    results,
                    vote_results,
                    final_player_rounds[key][0],
                    final_player_rounds[key][1]
                )

                reasonable_intent_data.append(reasonable_intent)
                form_con_intent_following_data.append(form_con_intent_following)
                refine_con_intent_following_data.append(refine_con_intent_following)
                complete_intent_following_data.extend(complete_intent_following)
    

In [27]:
def is_intent_following_good(row, to_retain_player_rounds=None, for_guessing=False):
    if to_retain_player_rounds:
        player, round, is_secondary_round = row['player'], row['round'], row['is_secondary_round']
        key = (player, round, is_secondary_round)
        # print(key)

        if key not in to_retain_player_rounds:
            return False

    intent_fol_think_1, intent_fol_speech_1 = row['intent_fol_think_1'], row['intent_fol_speech_1']
    intent_fol_think_2, intent_fol_speech_2 = row['intent_fol_think_2'], row['intent_fol_speech_2']

    form_check = True
    ref_check = True

    if not for_guessing:
        for ele in intent_fol_think_1.split(','):
            if ele == 'nan':
                # form_check = False
                break
            if ele != '':
                if int(ele) < 3:
                    form_check = False
        
        for ele in intent_fol_speech_1.split(','):
            if ele == 'nan':
                # form_check = False
                break
            if ele != '':
                if int(ele) < 3:
                    form_check = False

        for ele in intent_fol_think_2.split(','):
            if ele == 'nan':
                # ref_check = False
                break
            if ele != '':
                if int(ele) < 3:
                    ref_check = False

    for ele in intent_fol_speech_2.split(','):
        if ele == 'nan':
            # ref_check = False
            break
        if ele != '':
            if int(ele) < 3:
                ref_check = False

    if for_guessing:
        return ref_check
    return form_check or ref_check

In [28]:
def convert_ann_to_num_list(ann):
    res = []
    for ele in ann.split(','):
        if ele == 'nan':
            return []
        if ele == '':
            return []
        res.append(int(ele))
    return res

In [29]:
def filter_out_non_impactful_rows(evaluation_df, annotation_df):

    for_guessing = True

    if 'is_secondary_round' not in evaluation_df:
        evaluation_df['is_secondary_round'] = False

        found_player_round = {}
        # setting secondary round to True for second occurence of the key, so first one will be set to False
        for idx, row in evaluation_df.iterrows():
            player, round = row['player'], row['round']
            key = (player, round)
            if key in found_player_round:
                evaluation_df.at[idx, 'is_secondary_round'] = True
            else:
                found_player_round[key] = True
 
        to_retain_player_rounds = {}
        for idx, row in annotation_df.iterrows():
            player, round, is_secondary_round = row['player'], row['round'], row['is_secondary_round']
            key = (player, round, is_secondary_round)
            if key not in to_retain_player_rounds:
                to_retain_player_rounds[key] = True

        evaluation_df_filtered = evaluation_df[evaluation_df.apply(lambda row: is_intent_following_good(row, to_retain_player_rounds, for_guessing=for_guessing), axis=1)]
    else:
        evaluation_df_filtered = evaluation_df[evaluation_df.apply(lambda row: is_intent_following_good(row, for_guessing=for_guessing), axis=1)]

    final_player_rounds = {}
    for idx, row in evaluation_df_filtered.iterrows():
        player, round, is_secondary_round = row['player'], row['round'], row['is_secondary_round']
        intent_fol_think_1, intent_fol_speech_1 = convert_ann_to_num_list(row['intent_fol_think_1']), convert_ann_to_num_list(row['intent_fol_speech_1'])
        intent_fol_think_2, intent_fol_speech_2 = convert_ann_to_num_list(row['intent_fol_think_2']), convert_ann_to_num_list(row['intent_fol_speech_2'])
        key = (player, round, is_secondary_round)
        if key not in final_player_rounds:
            final_player_rounds[key] = ((intent_fol_think_1, intent_fol_speech_1), (intent_fol_think_2, intent_fol_speech_2))

    return final_player_rounds

In [34]:
def read_data(gameplay_folder):
    game_ts = gameplay_folder.split("_")[-1]
    gameplay_data, results = {}, {}
    with open(gameplay_folder + '/results.json') as f:
        results = json.load(f)

    gp_file_name = 'game_play_data.json'
    annotation_file = None
    for x in os.listdir(gameplay_folder):
        if x.startswith('game_play_data'):
            gp_file_name = x
        if x.startswith('empty-'):
            annotation_file = x
    with open(gameplay_folder + f'/{gp_file_name}') as f:
        gameplay_data = json.load(f)
        
    if not annotation_file:
        annotation_file = f'annotation-{game_ts}.csv'
    evaluation_df = pd.read_csv(gameplay_folder + f'/annotation-{game_ts}.csv')
    annotation_df = pd.read_csv(gameplay_folder + f'/{annotation_file}')

    evaluation_df.dropna(subset=['intent_selection'])
    
    evaluation_df['intent_selection'] = evaluation_df['intent_selection'].astype(str).apply(lambda x: x.replace('.', ','))
    evaluation_df['intent_fol_think_1'] = evaluation_df['intent_fol_think_1'].astype(str).apply(lambda x: x.replace('.', ','))
    evaluation_df['intent_fol_speech_1'] = evaluation_df['intent_fol_speech_1'].astype(str).apply(lambda x: x.replace('.', ','))
    evaluation_df['intent_selection_mod'] = evaluation_df['intent_selection_mod'].astype(str).apply(lambda x: x.replace('.', ','))
    evaluation_df['intent_fol_think_2'] = evaluation_df['intent_fol_think_2'].astype(str).apply(lambda x: x.replace('.', ','))
    evaluation_df['intent_fol_speech_2'] = evaluation_df['intent_fol_speech_2'].astype(str).apply(lambda x: x.replace('.', ','))
    return gameplay_data, results, evaluation_df, annotation_df

In [35]:
for folder in experiment_folders:
    print(folder)
    gameplay_data, results, evaluation_df, annotation_df = read_data(folder)
    final_player_rounds = filter_out_non_impactful_rows(evaluation_df, annotation_df)
    process_results(folder.split('/')[-1], gameplay_data, results, final_player_rounds)

output/gpt-3_5-turbo-games/annotated/our_annotations/game_2024-03-06-16-53-2
output/gpt-3_5-turbo-games/annotated/our_annotations/game_2024-03-06-16-21-47
output/gpt-3_5-turbo-games/annotated/our_annotations/game_2024-03-06-16-39-8
output/gpt-3_5-turbo-games/annotated/our_annotations/game_2024-03-06-16-21-42
output/gpt-3_5-turbo-games/annotated/our_annotations/game_2024-03-06-15-10-42


In [36]:
len(complete_intent_following_data)

124

round wise count

In [37]:
round_count = defaultdict(int)
for d in complete_intent_following_data:
    rnd = int(d['id'].split('_')[1].split('-')[-1])%10
    round_count[rnd] += 1

round_count

defaultdict(int, {2: 40, 3: 30, 1: 30, 4: 24})

In [38]:
sorted(round_count.items())

[(1, 30), (2, 40), (3, 30), (4, 24)]

In [39]:
len(refine_con_intent_following_data)

100

In [40]:
intent_count = 0
for data in refine_con_intent_following_data:
    intent_count += len(data['intents'])
intent_count

238

In [41]:
refine_con_intent_following_impactful_data = []
intent_count = 0
for data in refine_con_intent_following_data:
    intents = data['intents']
    filtered_intents = []
    for intent in intents:
        if intent not in impactful_intents:
            continue
        filtered_intents.append(intent)
    final_data = data.copy()
    final_data['intents'] = filtered_intents
    if len(filtered_intents) == 0:
        continue
    refine_con_intent_following_impactful_data.append(final_data)
    intent_count += len(filtered_intents)

intent_count

132

In [42]:
round_count = defaultdict(int)
for d in refine_con_intent_following_impactful_data:
    rnd = int(d['id'].split('-')[-1])%10
    round_count[rnd] += 1

round_count

defaultdict(int, {2: 26, 1: 17, 3: 23, 4: 17})

In [43]:
sorted(round_count.items())

[(1, 17), (2, 26), (3, 23), (4, 17)]

In [44]:
len(reasonable_intent_data)

100

In [45]:
len(refine_con_intent_following_impactful_data)

83

In [46]:
reasonable_intent_data[13].keys()

dict_keys(['id', 'context', 'intents', 'reason'])

In [47]:
complete_intent_following_data[13].keys()

dict_keys(['id', 'context', 'intents', 'think', 'speak'])

In [48]:
refine_con_intent_following_data[13].keys()

dict_keys(['id', 'context', 'intents', 'think', 'speak'])

In [49]:
complete_intent_following_data[0]

{'id': 'game_2024-03-06-16-53-2-player1-2_form',
 'context': "**Name**: Player1\n\n**Role**: Morgana\n\n**Role Details**: \nYou are Morgana. These are your goals and characteristics: \n- Pretend to be Merlin to mislead Percival and the good side.\n- Goal(Main Intent): Fail quests and Confuse and sabotage to win.\n- Assassin is Player3\n\n**Round**: 2\n\n**Current Leader**: Player2\n\n**Current Team**: Player1, Player4, Player5\n\n**Previous Rounds Team Voting**:\nRound 1: Player1 = agree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = disagree \nRound 2: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = agree \n\n**Previous Results**: \nRound 1: Team = Player1, Player2, Result = success\n\n**Previous Rounds Summary**:\nSummary questions: \n\nWho was the leader for this round?\nWhat was the proposed team for this round?\nWhy do you think the leader proposed the current team?\nDid everyone agree for the proposed team?\nIf some people dis

In [50]:
print(refine_con_intent_following_data[0]['context'])

**Name**: Player1

**Role**: Morgana

**Role Details**: 
You are Morgana. These are your goals and characteristics: 
- Pretend to be Merlin to mislead Percival and the good side.
- Goal(Main Intent): Fail quests and Confuse and sabotage to win.

**Round**: 2

**Player Name**: Player2

**Current Leader**: Player2

**Current Team**: Player1, Player4, Player5

**Previous Rounds Team Voting**:
Round 1: Player1 = agree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = disagree 
Round 2: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = agree, Player5 = agree 

**Previous Results**: 
Round 1: Team = Player1, Player2, Result = success

**Previous Rounds Summary**:
Summary questions: 

Who was the leader for this round?
What was the proposed team for this round?
Why do you think the leader proposed the current team?
Did everyone agree for the proposed team?
If some people disagreed, why do you think they disagreed?
Was the quest successful?
If the quest was succ

In [152]:
print(complete_intent_following_data[6]['context'])

**Name**: Player3

**Role**: Percival

**Role Details**: 
You are percival and you need to help Merlin. These are your goals and characteristics:
- Know identities of Merlin and Morgana, but unsure who is who.
- Use subtle hints to guide team and protect Merlin.
- Be cautious not to expose Merlin while deciphering true identities.
- Goal(Main Intent): Win while safeguarding Merlin.
- Player5 and Player2 are Merlin and Morgana, but you don't know which is Merlin and which is Morgana.

**Round**: 2

**Current Leader**: Player2

**Current Team**: Player2, Player3, Player5

**Previous Rounds Team Voting**:
Round 1: Player1 = agree, Player2 = agree, Player3 = agree, Player4 = agree, Player5 = disagree 
Round 2: Player1 = agree, Player2 = agree, Player3 = agree, Player4 = agree, Player5 = disagree 

**Previous Results**: 
Round 1: Team = Player1, Player2, Result = failure

**Previous Rounds Summary**:
Summary questions: 

Who was the leader for this round?
What was the proposed team for this

In [51]:
print(refine_con_intent_following_data[13]['intents'])

['pretend to have information and act like merlin', 'express you are a loyal player', 'support your teammate to be on the quest team']


In [154]:
print(complete_intent_following_data[13]['id'])

game_2024-03-29-1-27-24-player1-1_form


In [10]:
def save_data(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

In [52]:
SAVE_FOLDER = f"output/gpt-3_5-turbo-games/annotated/evaluation_our_annotations/structured_context_files"

In [53]:
os.makedirs(SAVE_FOLDER)

FileExistsError: [Errno 17] File exists: 'output/gpt-3_5-turbo-games/annotated/evaluation_our_annotations/structured_context_files'

In [54]:
SUFFIX = "_all"

In [72]:
# save_data(reasonable_intent_data, f"{SAVE_FOLDER}/reasonable_intent{SUFFIX}.json")
# save_data(form_con_intent_following_data, f"{SAVE_FOLDER}/frm_con_intent_following{SUFFIX}.json")
# save_data(refine_con_intent_following_data, f"{SAVE_FOLDER}/intent_guessing{SUFFIX}.json")
save_data(complete_intent_following_data, f"{SAVE_FOLDER}/intent_summarization{SUFFIX}.json")

In [57]:
save_data(refine_con_intent_following_data, f"{SAVE_FOLDER}/intent_guessing_second_order_5_games.json")

### saving as csv

In [319]:
len(refine_con_intent_following_data)

71

## Saving intent guessing data

In [320]:
refine_con_intent_following_df = pd.DataFrame(refine_con_intent_following_impactful_data)
refine_con_intent_following_df.head()

,id,context,intents,think,speak
0,game_2024-03-29-12-52-20-player1-1,**Name**: Player3\n\n**Role**: Percival\n\n**R...,[express you are a loyal player],"Ensuring my cover as the Assassin is critical,...","I propose that I, Player1, and Player2 embark ..."
1,game_2024-03-29-12-52-20-player1-5,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,[question the leader why they selected a parti...,I must continue to be cautious with my wording...,"Player5, your selection is intriguing; could y..."
2,game_2024-03-29-12-52-20-player2-3,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,[express concerns about a player from a failed...,"As Morgana, my actions should create uncertain...",It is curious that Player3 has not chosen to b...
3,game_2024-03-29-12-52-20-player2-4,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[question the leader why they selected a parti...,I need to maintain the facade of being cautiou...,"Player4, I find your choice intriguing. In the..."
4,game_2024-03-29-12-52-20-player2-5,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[question the leader why they selected a parti...,My primary objective is to ensure the failure ...,"Player5, your choice of team members is crucia..."


In [321]:
refine_con_intent_following_df = refine_con_intent_following_df.drop(['think'], axis=1)
refine_con_intent_following_df.head()

,id,context,intents,speak
0,game_2024-03-29-12-52-20-player1-1,**Name**: Player3\n\n**Role**: Percival\n\n**R...,[express you are a loyal player],"I propose that I, Player1, and Player2 embark ..."
1,game_2024-03-29-12-52-20-player1-5,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,[question the leader why they selected a parti...,"Player5, your selection is intriguing; could y..."
2,game_2024-03-29-12-52-20-player2-3,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,[express concerns about a player from a failed...,It is curious that Player3 has not chosen to b...
3,game_2024-03-29-12-52-20-player2-4,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[question the leader why they selected a parti...,"Player4, I find your choice intriguing. In the..."
4,game_2024-03-29-12-52-20-player2-5,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[question the leader why they selected a parti...,"Player5, your choice of team members is crucia..."


In [322]:
refine_con_intent_following_df.to_csv(f"{SAVE_FOLDER}/intent_guessing{SUFFIX}_impactful.csv", index=False)

In [323]:
len(refine_con_intent_following_df)

62

In [324]:
refine_con_intent_following_wihtout_intents_df = refine_con_intent_following_df.drop(['intents'], axis=1)
refine_con_intent_following_wihtout_intents_df.head()

,id,context,speak
0,game_2024-03-29-12-52-20-player1-1,**Name**: Player3\n\n**Role**: Percival\n\n**R...,"I propose that I, Player1, and Player2 embark ..."
1,game_2024-03-29-12-52-20-player1-5,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,"Player5, your selection is intriguing; could y..."
2,game_2024-03-29-12-52-20-player2-3,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,It is curious that Player3 has not chosen to b...
3,game_2024-03-29-12-52-20-player2-4,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"Player4, I find your choice intriguing. In the..."
4,game_2024-03-29-12-52-20-player2-5,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"Player5, your choice of team members is crucia..."


In [325]:
refine_con_intent_following_wihtout_intents_df.to_csv(f"{SAVE_FOLDER}/intent_guessing_without_intent{SUFFIX}_impactful.csv", index=False)

In [580]:
refine_con_intent_following_df_50 = refine_con_intent_following_df.sample(n=50)
len(refine_con_intent_following_df_50)

50

In [581]:
refine_con_intent_following_df_50.to_csv(f"{SAVE_FOLDER}/intent_guessing_gpt4_50.csv", index=False)

In [582]:
refine_con_intent_following_df_50 = refine_con_intent_following_df_50.drop(['intents'], axis=1)
refine_con_intent_following_df_50.head()

,id,context,speak
10,game_2024-03-29-12-52-20-player3-4,**Name**: Player4\n\n**Role**: Servant\n\n**Ro...,"Team, our previous quest outcomes must guide o..."
49,game_2024-03-29-1-59-14-player4-2,**Name**: Player5\n\n**Role**: Assassin\n\n**R...,"Player2, I understand the need for a strong te..."
36,game_2024-03-29-1-27-24-player5-3,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"Player3, I find it risky to include a member f..."
62,game_2024-03-29-0-16-48-player2-4,**Name**: Player3\n\n**Role**: Assassin\n\n**R...,"While I trust Player4's judgment, the inclusio..."
26,game_2024-03-29-1-27-24-player2-3,**Name**: Player3\n\n**Role**: Morgana\n\n**Ro...,Player3's decision to include Player1 again is...


In [583]:
refine_con_intent_following_df_50.to_csv(f"{SAVE_FOLDER}/intent_guessing_gpt4_without_intents_50.csv", index=False)

In [584]:
refine_con_intent_following_data_50 = []
sampled_50_ref_ids = {id: True for id in refine_con_intent_following_df_50['id'].values.tolist()}
for data in refine_con_intent_following_data:
    id = data['id']
    if id in sampled_50_ref_ids:
        refine_con_intent_following_data_50.append(data)

len(refine_con_intent_following_data_50)

50

In [585]:
save_data(refine_con_intent_following_data_50, f"{SAVE_FOLDER}/intent_guessing_gpt4_50.json")

## Saving summarization data

In [75]:
complete_intent_following_df = pd.DataFrame(complete_intent_following_data)
complete_intent_following_df.head()

,id,context,intents,think,speak
0,game_2024-03-29-12-52-20-player1-1_form,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[stay hidden in discussions and act like a loy...,"As the assassin, my main objective is to sow d...","For this first quest, I believe it's crucial t..."
1,game_2024-03-29-12-52-20-player1-2_ref,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[defend your teammate against allegations that...,To maintain my cover and push the game towards...,"I trust Player2's judgment for this quest, and..."
2,game_2024-03-29-12-52-20-player1-5_form,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,[question the leader why they selected a parti...,I need to cast suspicion subtly to protect my ...,"I'm curious, Player5, about your rationale for..."
3,game_2024-03-29-12-52-20-player2-2_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,[pretend to have information and act like merl...,I need to subtly suggest that I have insider k...,I have a strong feeling about the loyalty of P...
4,game_2024-03-29-12-52-20-player2-3_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,[question the leader why they didnt put themse...,"As Morgana, my sub-goals are to cast suspicion...",I find it a bit unusual that Player3 hasn't pr...


In [76]:
complete_intent_following_df.to_csv(f"{SAVE_FOLDER}/intent_summarization{SUFFIX}.csv", index=False)

In [77]:
complete_intent_following_wihtout_intents_df = complete_intent_following_df.drop(['intents'], axis=1)
complete_intent_following_wihtout_intents_df.head()

,id,context,think,speak
0,game_2024-03-29-12-52-20-player1-1_form,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,"As the assassin, my main objective is to sow d...","For this first quest, I believe it's crucial t..."
1,game_2024-03-29-12-52-20-player1-2_ref,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,To maintain my cover and push the game towards...,"I trust Player2's judgment for this quest, and..."
2,game_2024-03-29-12-52-20-player1-5_form,**Name**: Player1\n\n**Role**: Assassin\n\n**R...,I need to cast suspicion subtly to protect my ...,"I'm curious, Player5, about your rationale for..."
3,game_2024-03-29-12-52-20-player2-2_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,I need to subtly suggest that I have insider k...,I have a strong feeling about the loyalty of P...
4,game_2024-03-29-12-52-20-player2-3_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,"As Morgana, my sub-goals are to cast suspicion...",I find it a bit unusual that Player3 hasn't pr...


In [78]:
complete_intent_following_wihtout_intents_df.to_csv(f"{SAVE_FOLDER}/intent_summarization_without_intent{SUFFIX}.csv", index=False)

In [493]:
complete_intent_following_df_100 = complete_intent_following_df.sample(n=50)
len(complete_intent_following_df_100)

50

In [494]:
complete_intent_following_df_100.to_csv(f"{SAVE_FOLDER}/intent_summarization_gpt4_50.csv", index=False)

In [495]:
complete_intent_following_without_intents_df_100 = complete_intent_following_df_100.drop(['intents'], axis=1)
complete_intent_following_without_intents_df_100.head()

,id,context,think,speak
34,game_2024-03-29-0-16-48-player4-5_form,**Name**: Player4\n\n**Role**: Percival\n\n**R...,I need to focus on protecting Merlin's identit...,"Player5, your selection intrigues me; could yo..."
0,game_2024-03-29-1-59-14-player1-2_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must use the information I have a...","I find the team selection intriguing, Player2...."
19,game_2024-03-29-0-16-48-player1-3_form,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,"As Merlin, I must cast doubt on the evil playe...","Player3, as the leader, could you share the in..."
44,game_2024-03-29-0-51-34-player2-2_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,"As Morgana, my first sub-goal is to emulate Me...","Fellow knights, based on our discussions and t..."
5,game_2024-03-29-1-59-14-player2-3_form,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,I need to subtly undermine the trust in the gr...,"Player3, your choice of including Player4 is i..."


In [496]:
complete_intent_following_without_intents_df_100.to_csv(f"{SAVE_FOLDER}/intent_summarization_gpt4_without_intent_50.csv", index=False)

In [497]:
complete_intent_following_data_100 = []
sampled_100_ids = {id: True for id in complete_intent_following_df_100['id'].values.tolist()}
for data in complete_intent_following_data:
    id = data['id']
    if id in sampled_100_ids:
        complete_intent_following_data_100.append(data)

len(complete_intent_following_data_100)

50

In [498]:
save_data(complete_intent_following_data_100, f"{SAVE_FOLDER}/intent_summarization_gpt4_50.json")

### convert json to csv

In [39]:
intent_results = {}
with open('evaluation/ctx_with_discussions/results/intent_guessing-2024-03-06-16-21-47.json') as f:
        intent_results = json.load(f)

In [40]:
intent_results

{'game_2024-03-06-16-21-47-player1-1': {'id': 'game_2024-03-06-16-21-47-player1-1',
  'context': "**Name**: Player1\n\n**Role**: Merlin\n\n**Role Details**: \nYou are Merlin. These are your goals and characteristics: \n- Know the identities of evil players.\n- Subtly guide your team, especially Percival.\n- Avoid behaviours that expose your role: overly accusing, being too helpful.\n- Goal(Main Intent): Win without revealing identity.\n\n**Other Players**: Player2 = Morgana, Player3 = Servant, Player4 = Percival, Player5 = Assassin\n\n**Round**: 1\n\n**Current Leader**: Player2\n\n**Current Team**: Player1, Player2\n\n**Previous Rounds Team Voting**:\nNo previous team votes. This is the first round.\n\n**Previous Results**: \nNo previous results.\n\n**Previous Rounds Summary**:\nSummary questions: \n\nWho was the leader for this round?\nWhat was the proposed team for this round?\nWhy do you think the leader proposed the current team?\nDid everyone agree for the proposed team?\nIf some 

In [41]:
len(intent_results)

20

In [42]:
intent_results.keys()

dict_keys(['game_2024-03-06-16-21-47-player1-1', 'game_2024-03-06-16-21-47-player1-2', 'game_2024-03-06-16-21-47-player1-3', 'game_2024-03-06-16-21-47-player1-4', 'game_2024-03-06-16-21-47-player2-1', 'game_2024-03-06-16-21-47-player2-2', 'game_2024-03-06-16-21-47-player2-3', 'game_2024-03-06-16-21-47-player2-4', 'game_2024-03-06-16-21-47-player3-1', 'game_2024-03-06-16-21-47-player3-2', 'game_2024-03-06-16-21-47-player3-3', 'game_2024-03-06-16-21-47-player3-4', 'game_2024-03-06-16-21-47-player4-1', 'game_2024-03-06-16-21-47-player4-2', 'game_2024-03-06-16-21-47-player4-3', 'game_2024-03-06-16-21-47-player4-4', 'game_2024-03-06-16-21-47-player5-1', 'game_2024-03-06-16-21-47-player5-2', 'game_2024-03-06-16-21-47-player5-3', 'game_2024-03-06-16-21-47-player5-4'])

In [43]:
intent_results_df = pd.DataFrame(intent_results.values())
intent_results_df.head()

,id,context,intents,reason,guessed_intents
0,game_2024-03-06-16-21-47-player1-1,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,1. Question the leader why they selected a par...,I chose these intents to question the leader's...,"[Select team composition, Defend team selection]"
1,game_2024-03-06-16-21-47-player1-2,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,1. Share concerns about an evil player\n2. Cou...,I chose these intents to subtly guide the team...,"[Support team proposal, Question the leader wh..."
2,game_2024-03-06-16-21-47-player1-3,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,1. Support team proposal\n2. Support one loyal...,I chose these intents to subtly guide the team...,"[Support team proposal, Strongly convince the ..."
3,game_2024-03-06-16-21-47-player1-4,**Name**: Player1\n\n**Role**: Merlin\n\n**Rol...,1. Express concerns about a player from a fail...,These intents focus on ensuring the team compo...,[Support loyal players of the previous quest t...
4,game_2024-03-06-16-21-47-player2-1,**Name**: Player2\n\n**Role**: Morgana\n\n**Ro...,1. Pretend to have information and act like Me...,I chose these intents to create confusion amon...,"[Express disagreement if not on team, Question..."


In [44]:
intent_results_df.iloc[4].values

array(['game_2024-03-06-16-21-47-player2-1',
       '**Name**: Player2\n\n**Role**: Morgana\n\n**Role Details**: \nYou are Morgana. These are your goals and characteristics: \n- Pretend to be Merlin to mislead Percival and the good side.\n- Goal(Main Intent): Fail quests and Confuse and sabotage to win.\n\n**Other Players**: Player1 = Merlin, Player3 = Servant, Player4 = Percival, Player5 = Assassin\n\n**Round**: 1\n\n**Current Leader**: Player2\n\n**Current Team**: \n\n**Previous Rounds Team Voting**:\nNo previous team votes. This is the first round.\n\n**Previous Results**: \nNo previous results.\n\n**Previous Rounds Summary**:\nSummary questions: \n\nWho was the leader for this round?\nWhat was the proposed team for this round?\nWhy do you think the leader proposed the current team?\nDid everyone agree for the proposed team?\nIf some people disagreed, why do you think they disagreed?\nWas the quest successful?\nIf the quest was successful, was any evil player part of the team? If ye

In [45]:
intent_results_df.to_csv('evaluation/ctx_with_discussions/results/intent_guessing-2024-03-06-16-21-47.csv')

### Intent Guessing context generation with perspective and example

In [243]:
with open(f'output/gpt-3_5-turbo-games/annotated/evaluation/guessing_data/intent_guessing{SUFFIX}_impactful.json', 'r') as f:
    intent_guessing_data = json.loads(f.read())

In [245]:
with open(f'output/gpt-3_5-turbo-games/annotated/evaluation/summarization_data/intent_summarization{SUFFIX}.json', 'r') as f:
    intent_summarization_data = json.loads(f.read())

In [246]:
len(intent_guessing_data), len(intent_summarization_data)

(864, 1299)

In [260]:
intent_summarization_impactful_data = []
intent_count = 0
for data in intent_summarization_data:
    intents = data['intents']
    filtered_intents = []
    for intent in intents:
        if intent not in impactful_intents:
            continue
        filtered_intents.append(intent)
    final_data = data.copy()
    final_data['intents'] = filtered_intents
    if len(filtered_intents) == 0:
        continue
    intent_summarization_impactful_data.append(final_data)
    intent_count += len(filtered_intents)

intent_count

1776

In [262]:
intent_summarization_data = intent_summarization_impactful_data

In [263]:
intent_guessing_data[0]

{'id': 'game_2024-03-14-16-27-7-player1-2',
 'context': '**Name**: Player2\n\n**Role**: Servant\n\n**Role Details**: \nYou are Loyal Servant of Arthur. These are your goals and characteristics: \n- No special knowledge, rely on discussion and voting.\n- Contribute to the success of Quests\n- Goal(Main Intent): Win by helping complete Quests and protecting Merlin.\n- \n\n\n**Round**: 2\n\n**Speaker Name**: Player1\n\n**Current Leader**: Player4\n\n**Current Team**: Player1, Player2, Player4\n\n**Previous Rounds Team Voting**:\nRound 1: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = disagree, Player5 = disagree (players didn"t agree on team)\nRound 1: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = disagree, Player5 = disagree \nRound 2: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = agree, Player5 = disagree (players didn"t agree on team)\nRound 2: Player1 = disagree, Player2 = disagree, Player3 = agree, Player4 = disagree, Play

In [264]:
intent_guessing_dict = {}
for sample in intent_guessing_data:
    data = sample
    id_ele = sample['id'].split('-')
    player = id_ele[-2]
    game_id = '-'.join(id_ele[:-2])

    data['game_id'] = game_id
    data['player'] = player
    intent_guessing_dict[sample['id']] = data

In [265]:
intent_summarization_dict = {}
for sample in intent_summarization_data:
    data = sample
    id_ele = sample['id'].split('-')
    game_id = '-'.join(id_ele[:-2])
    player = data['context'].split('\n')[0].split(': ')[-1].strip().lower()
    

    data['game_id'] = game_id
    data['player'] = player
    intent_summarization_dict[sample['id']] = data

In [266]:
len(intent_guessing_dict), len(intent_summarization_dict)

(864, 1091)

In [267]:
print(intent_guessing_data[0]['context'].find('**Round**:'))

306


In [268]:
print(intent_guessing_data[1]['context'])

**Name**: Player2

**Role**: Servant

**Role Details**: 
You are Loyal Servant of Arthur. These are your goals and characteristics: 
- No special knowledge, rely on discussion and voting.
- Contribute to the success of Quests
- Goal(Main Intent): Win by helping complete Quests and protecting Merlin.
- 


**Round**: 2

**Speaker Name**: Player1

**Current Leader**: Player3

**Current Team**: 

**Previous Rounds Team Voting**:
Round 1: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = disagree, Player5 = disagree (players didn"t agree on team)
Round 1: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = disagree, Player5 = disagree 
Round 2: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = agree, Player5 = disagree (players didn"t agree on team)
Round 2: Player1 = disagree, Player2 = disagree, Player3 = agree, Player4 = disagree, Player5 = disagree 

**Previous Results**: 
Round 1: Team = Player2, Player4, Result = failure

**Previous Rou

In [269]:
intent_sum_game_grouped = defaultdict(list)
for key, val in intent_summarization_dict.items():
    intent_sum_game_grouped[val['game_id']].append(val)

In [270]:
len(intent_sum_game_grouped)

40

In [271]:
intent_guessing_perspective_data = []
for key, val in intent_guessing_dict.items():
    game_id = val['game_id']
    player = val['player']
    sum_data = intent_sum_game_grouped[game_id]
    cur_idx = 0
    while sum_data[cur_idx]['player'] == player:
        cur_idx += 1

    perspective_player = sum_data[cur_idx]['player']
    perspective_player_role_details = sum_data[cur_idx]['context'][:sum_data[cur_idx]['context'].find('**Round**:')]

    intent_guess_context = val['context'][val['context'].find('**Round**:'):]

    context = perspective_player_role_details + '\n' + intent_guess_context
    persp_player_intents = sum_data[cur_idx]['intents']
    persp_player_think = sum_data[cur_idx]['think']
    persp_player_speak = sum_data[cur_idx]['speak']
    
    val['context'] = context
    val['persp_player_intents'] = persp_player_intents
    val['persp_player_think'] = persp_player_think
    val['persp_player_speak'] = persp_player_speak
    val['persp_player'] = perspective_player

    intent_guessing_perspective_data.append(val)
    

In [273]:
len(intent_summarization_data)

1091

In [274]:
len(intent_guessing_perspective_data)

864

In [275]:
intent_guessing_perspective_data[0]

{'id': 'game_2024-03-14-16-27-7-player1-2',
 'context': '**Name**: Player2\n\n**Role**: Servant\n\n**Role Details**: \nYou are Loyal Servant of Arthur. These are your goals and characteristics: \n- No special knowledge, rely on discussion and voting.\n- Contribute to the success of Quests\n- Goal(Main Intent): Win by helping complete Quests and protecting Merlin.\n- \n\n\n**Round**: 2\n\n**Speaker Name**: Player1\n\n**Current Leader**: Player4\n\n**Current Team**: Player1, Player2, Player4\n\n**Previous Rounds Team Voting**:\nRound 1: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = disagree, Player5 = disagree (players didn"t agree on team)\nRound 1: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = disagree, Player5 = disagree \nRound 2: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = agree, Player5 = disagree (players didn"t agree on team)\nRound 2: Player1 = disagree, Player2 = disagree, Player3 = agree, Player4 = disagree, Play

In [276]:
print(intent_guessing_perspective_data[0]['context'])

**Name**: Player2

**Role**: Servant

**Role Details**: 
You are Loyal Servant of Arthur. These are your goals and characteristics: 
- No special knowledge, rely on discussion and voting.
- Contribute to the success of Quests
- Goal(Main Intent): Win by helping complete Quests and protecting Merlin.
- 


**Round**: 2

**Speaker Name**: Player1

**Current Leader**: Player4

**Current Team**: Player1, Player2, Player4

**Previous Rounds Team Voting**:
Round 1: Player1 = disagree, Player2 = agree, Player3 = disagree, Player4 = disagree, Player5 = disagree (players didn"t agree on team)
Round 1: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = disagree, Player5 = disagree 
Round 2: Player1 = agree, Player2 = disagree, Player3 = disagree, Player4 = agree, Player5 = disagree (players didn"t agree on team)
Round 2: Player1 = disagree, Player2 = disagree, Player3 = agree, Player4 = disagree, Player5 = disagree 

**Previous Results**: 
Round 1: Team = Player2, Player4, Result 

In [277]:
save_data(intent_guessing_perspective_data, f"output/gpt-3_5-turbo-games/annotated/evaluation/guessing_data/intent_guessing{SUFFIX}_impactful_reflective.json")

## Hallucination detection data

In [2]:
with open(f'output/gpt-3_5-turbo-games/annotated/evaluation/guessing_data/intent_guessing.json', 'r') as f:
    intent_guessing_data = json.loads(f.read())

In [4]:
len(intent_guessing_data)

127

In [6]:
def filter_out_rows(data):
    keep = []
    for row in data:
        if 'express concerns about a player from a failed quest team and suggest to not include them in the current team' in row['intents']:
            keep.append(row)

    return keep

In [7]:
filtered_intent_guessing_data = filter_out_rows(intent_guessing_data)
len(filtered_intent_guessing_data)

43

In [11]:
save_data(filtered_intent_guessing_data, f"hallucination_detection_gpt_3-5.json")